In [1]:
import os
os.chdir('..')

In [2]:
from src.api.workflows.FAQs.extract_FAQs import obtener_faqs, obtener_faqs_unir
faqs = obtener_faqs(); faqs_unir = obtener_faqs_unir()

In [11]:
from src.api.common.dependency_container import DependencyContainer
from IPython.display import HTML  

DC = DependencyContainer(); DC.initialize()
pregunta = 'perrito, gatico, lobito'
request = {'Pregunta': pregunta, 'FAQs': faqs + faqs_unir}
response = DependencyContainer.get_faqs_workflow().execute(request)
max_faq = max(response['FAQs'], key=lambda x: float(x['Score']))  

ERROR:logger:Error in call_api using prompt: faqs_generator
Error code: 400 - {'error': {'message': "This model's maximum context length is 16385 tokens. However, you requested 16810 tokens (12714 in the messages, 4096 in the completion). Please reduce the length of the messages or completion.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
response

{'FAQs': [{'FAQ 1': '1. No puedo acceder con mis datos de usuario',
   'Contenido': '<div><p>Asegúrate en primer lugar que introduces bien los datos:</p><ul>\n<li><strong>Nombre de usuario:</strong> tu correo electrónico. En el ejemplo es "lidianarpaz@yahoo.com": sería un error poner un espacio más "lidianarpaz@yahoo.com " al final, o en medio o empezar con mayúscula. </li>\n<li><strong>Contraseña:</strong> si no fuiste tú quien eligió la contraseña, seguramente será la primera parte de tu correo electrónico, previa a la @. En el ejemplo es "lidianarpaz". Lo mismo: sería un error escribir un espacio más antes o después, o incluir una mayúscula. A veces es mejor copiar y pegar la contraseña. Si la contraseña la elegiste tú, sigue las mismas indicaciones en lo que se refiere a evitar espacios en blanco.</li>\n</ul><p class="centrar"><img alt="Datos de usuario" class="responsive" src="https://becat.online/FAQ/img/datos-usuario.png"/></p><p>El acceso al curso está en la sección "Mis cursos

In [15]:
faqs_salida = ''
for faq in response['FAQs']: 
    faqs_salida+=f"<h2>{list(faq.values())[0]}</h2>{faq['Contenido']}"
HTML(faqs_salida)